# Segmenting and Clustering Neighborhoods in Toronto

## Check install beautifulsoup

In [5]:
!pip install beautifulsoup4

Check install of lxml

In [6]:
!pip install lxml

Check requests library

In [7]:
!pip install requests

## Import and Process Website Data

In [131]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Set up data frame
PC_table_data = pd.DataFrame()

#Obtain data object from the website using the lxml parser
source = requests.get ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

#Find the postcode table in the text
Postcode_table = soup.find('table',{'class':'wikitable sortable'})

#Extract the table headings
table_headings = []
for heading_tag in Postcode_table.findAll ('th'):
    table_headings.append(heading_tag.text.replace('\n',' ').strip())

#Set up the list of postcodes, boroughs and districts
#The extraction approach here is dependent upon the strcuture of the data
#It assumes that the table is a series of rows with three data elements
PC_table_list = []
#For each table row extract the data
for row_tag in Postcode_table.findAll ('tr'):
    table_row = []
    #For each data element in the row extract the text removing newlines and spaces
    for data_tag in row_tag.findAll ('td'):
        table_row.append(data_tag.text.replace('\n',' ').strip())
    #Process the table rows to remove districts that are not assigned and to
    #copy the borough to the district if not assigned
    #build the table list
    if table_row:
        if (table_row[2] == 'Not assigned') and (table_row[1] == 'Not assigned'):
            # Do nothing
            pass
        elif (table_row[2] == 'Not assigned') and (table_row[1] != 'Not assigned'):
            table_row [2] = table_row [1]
            PC_table_list.append(table_row)
        else:
            PC_table_list.append(table_row)

#Put the extracted headings into the panda frame
PC_table_data = pd.DataFrame(columns = table_headings)
#Append the table list to teh panda data frame
PC_table_data = PC_table_data.append(pd.DataFrame(PC_table_list, columns=table_headings),ignore_index=True)

#Group the panda frame by postcode and concatanate the neighbourhoods
PC_table_grouped = PC_table_data.groupby(['Postcode', 'Borough']).agg({'Neighbourhood':lambda x:', '.join(x)}).reset_index()

#Display the grouped panda frame
PC_table_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [132]:
#Display the shape of the grouped panda frame
PC_table_grouped.shape

(103, 3)

# Obtain Latitude and Longitude for the Postcodes

## We initially work with postcodes

In [133]:
#geocodex solution was unusable
#Read list of latitude and longitudes from file
#Note that there is no assumption about the order
#of the postcodes in the file so teh code is somewhat
#ineffcient

#Read file
postcode_file = pd.read_csv('https://cocl.us/Geospatial_data')

#Set up latitude and longitude lists
lat_list = []
lng_list = []
#Compare post codes and append lists
for i in range(0, len(PC_table_grouped)):
    for j in range(0, len(postcode_file)):
        if (PC_table_grouped.iloc[i]['Postcode'] == postcode_file.iloc[j][0]):
            lat_list.append(postcode_file.iloc[j][1])
            lng_list.append(postcode_file.iloc[j][2])

#Add lists to panda frame
PC_table_grouped['Latitude'] = lat_list
PC_table_grouped['Longitude'] = lng_list
            

In [134]:
#Check new table
PC_table_grouped.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Visualise the Data

In [5]:
!conda install -c conda-forge folium=0.5.0 --yes #Install Folium



Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [135]:
from geopy.geocoders import Nominatim
import folium # map rendering library


address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [136]:
#Show map of Toronto
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# display map
toronto_map

Add postcodes

In [137]:
# add the postcodes as blue circle markers
postcode_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for i in range(0, len(PC_table_grouped)):
    lat = PC_table_grouped['Latitude'][i]
    lng = PC_table_grouped['Longitude'][i]
    label = PC_table_grouped['Postcode'][i]
    #for lat, lng, label in zip(PC_table_grouped['Latitude'], PC_table_grouped['Longitude'], PC_table_grouped['Neighbourhood']):
    #print(lat,lng,label)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(postcode_map)

# display map
postcode_map

## Filter Boroughs that Contain Toronto

In [138]:
district_toronto=[]
for i in range(0, len(PC_table_grouped)):
    district_toronto.append('Toronto' in PC_table_grouped['Borough'][i])

#PC_table_grouped = PC_table_grouped[district_toronto].reset_index(drop=True)
#PC_table_grouped = PC_table_grouped[district_toronto].reset_index()
PC_table_grouped = PC_table_grouped[district_toronto].reset_index(drop=True)
PC_table_grouped

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


# Display Map With Only Toronoto in the Borough

In [139]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# add the boroughs as blue circle markers
borough_map = folium.Map(location=[latitude, longitude], zoom_start=13)

for i in range(0, len(PC_table_grouped)):
    lat = PC_table_grouped['Latitude'][i]
    lng = PC_table_grouped['Longitude'][i]
    label = PC_table_grouped['Postcode'][i]
    #for lat, lng, label in zip(PC_table_grouped['Latitude'], PC_table_grouped['Longitude'], PC_table_grouped['Neighbourhood']):
    #print(lat,lng,label)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(borough_map)

# display map
borough_map



## Define Foursquare credentials

In [140]:
CLIENT_ID = '4HAINK3PYXA1XG2LGU5VARUNWZOAABPSPADBDJIU3DHYNPXV' # your Foursquare ID
CLIENT_SECRET = 'L20HW0TY51QAOQ54CSPJPXP2YTRPTLZX55KRKLDQH4THG4AH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4HAINK3PYXA1XG2LGU5VARUNWZOAABPSPADBDJIU3DHYNPXV
CLIENT_SECRET:L20HW0TY51QAOQ54CSPJPXP2YTRPTLZX55KRKLDQH4THG4AH


In [141]:
PC_table_grouped.loc[0, 'Borough']

'East Toronto'

## Increase radius to 1000m to include more venues
It was found that the clustering did not have enough venues if
500m was used.

In [143]:
#Getting the top 100 venues withon 500 metres of this part of East Toronto

LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    PC_table_grouped.loc[0,'Latitude'], 
    PC_table_grouped.loc[0,'Longitude'], 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d725e7a724750002cf6f951'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 75,
  'suggestedBounds': {'ne': {'lat': 43.685357409000005,
    'lng': -79.28061062898105},
   'sw': {'lat': 43.66735739099998, 'lng': -79.30545177101895}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f5a855be4b0a4baa1ae0063',
       'name': "Tori's Bakeshop",
       'location': {'address': '2188 Queen Street E',
        'crossStreet': 'Balsam Ave',
        'lat': 43.672113947269565,
        'lng': -79.29033140068843,
        'labeledLatLngs

In [144]:
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
1,The Beech Tree,Gastropub,43.680493,-79.288846
2,The Fox Theatre,Indie Movie Theater,43.672801,-79.287272
3,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993
4,Glen Manor Ravine,Trail,43.676821,-79.293942


In [145]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

75 venues were returned by Foursquare.


# Repeat the Approach Across the Boroughs

## Create a function to repeat the same process to all the Toronto boroughs

In [146]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Repeat the above function on each borough and create a new dataframe called toronto_venues.

In [147]:
# Repeat for Toronto venues across postcodes
# If Boroughs are used here it collapses down to 4 making clustering impossible

toronto_venues = getNearbyVenues(names=PC_table_grouped['Postcode'],
                                   latitudes=PC_table_grouped['Latitude'],
                                   longitudes=PC_table_grouped['Longitude']
                                  )




M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7Y


## Size of resulting data frame

In [148]:
print(toronto_venues.shape)
toronto_venues.head()

(3086, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,M4E,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
2,M4E,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
3,M4E,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail


## Let's check how many venues were returned for each neighbourhood

In [149]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M4E,75,75,75,75,75,75
M4K,100,100,100,100,100,100
M4L,80,80,80,80,80,80
M4M,100,100,100,100,100,100
M4N,9,9,9,9,9,9
M4P,100,100,100,100,100,100
M4R,46,46,46,46,46,46
M4S,100,100,100,100,100,100
M4T,59,59,59,59,59,59


## Let's find out how many unique categories can be curated from all the returned venues

In [150]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 278 uniques categories.


# Analyze Each Neighborhood

In [151]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Examine Data Frame Size

In [152]:
toronto_onehot.shape

(3086, 278)

## Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [153]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Zoo,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,...,0.013333,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0000
1,M4K,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0200
2,M4L,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0125,0.0125
3,M4M,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.01,...,0.010000,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0000,0.0100
4,M4N,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,...,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0000


In [154]:
toronto_grouped.shape

(38, 278)

## Print each neighborhood along with the top 5 most common venues

In [155]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
         venue  freq
0          Pub  0.07
1  Coffee Shop  0.07
2  Pizza Place  0.04
3         Park  0.04
4        Beach  0.04


----M4K----
                venue  freq
0    Greek Restaurant  0.12
1         Coffee Shop  0.08
2                Café  0.06
3                 Pub  0.05
4  Italian Restaurant  0.03


----M4L----
               venue  freq
0  Indian Restaurant  0.09
1        Coffee Shop  0.06
2               Café  0.04
3              Beach  0.04
4               Park  0.02


----M4M----
                 venue  freq
0          Coffee Shop  0.08
1  American Restaurant  0.05
2                  Bar  0.05
3               Bakery  0.05
4                 Café  0.04


----M4N----
                  venue  freq
0                  Park  0.11
1  Gym / Fitness Center  0.11
2              Bus Line  0.11
3             Bookstore  0.11
4          College Quad  0.11


----M4P----
                  venue  freq
0           Coffee Shop  0.12
1    Italian Restaurant  0.05
2  Fast Food Resta

## Put that into a pandas dataframe

In [156]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Create the new dataframe and display the top 10 venues for each neighborhood.

In [157]:
import numpy as np # library to handle data in a vectorized manner

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Pub,Coffee Shop,Japanese Restaurant,Pizza Place,Bar,Breakfast Spot,Beach,Park,Caribbean Restaurant,Bakery
1,M4K,Greek Restaurant,Coffee Shop,Café,Pub,Pizza Place,Italian Restaurant,Fast Food Restaurant,Discount Store,Diner,Bakery
2,M4L,Indian Restaurant,Coffee Shop,Café,Beach,Bakery,Burrito Place,Butcher,Park,Gym,Brewery
3,M4M,Coffee Shop,American Restaurant,Bar,Bakery,Café,Brewery,Vietnamese Restaurant,Italian Restaurant,French Restaurant,Diner
4,M4N,College Quad,Bookstore,College Gym,Coffee Shop,Café,Gym / Fitness Center,Trail,Bus Line,Park,Eastern European Restaurant


# Cluster Neighborhoods

In [158]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 4, 0, 4, 4, 4, 4, 4], dtype=int32)

In [159]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Pub,Coffee Shop,Japanese Restaurant,Pizza Place,Bar,Breakfast Spot,Beach,Park,Caribbean Restaurant,Bakery
1,M4K,Greek Restaurant,Coffee Shop,Café,Pub,Pizza Place,Italian Restaurant,Fast Food Restaurant,Discount Store,Diner,Bakery
2,M4L,Indian Restaurant,Coffee Shop,Café,Beach,Bakery,Burrito Place,Butcher,Park,Gym,Brewery
3,M4M,Coffee Shop,American Restaurant,Bar,Bakery,Café,Brewery,Vietnamese Restaurant,Italian Restaurant,French Restaurant,Diner
4,M4N,College Quad,Bookstore,College Gym,Coffee Shop,Café,Gym / Fitness Center,Trail,Bus Line,Park,Eastern European Restaurant


## Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
## Note - becasue the original dataframe used postcodes we need to switch headings for the join to work

In [160]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

PC_table_grouped = PC_table_grouped.rename(columns={'Postcode': 'Neighborhood'})
toronto_merged = PC_table_grouped

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Coffee Shop,Japanese Restaurant,Pizza Place,Bar,Breakfast Spot,Beach,Park,Caribbean Restaurant,Bakery
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4,Greek Restaurant,Coffee Shop,Café,Pub,Pizza Place,Italian Restaurant,Fast Food Restaurant,Discount Store,Diner,Bakery
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Indian Restaurant,Coffee Shop,Café,Beach,Bakery,Burrito Place,Butcher,Park,Gym,Brewery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Coffee Shop,American Restaurant,Bar,Bakery,Café,Brewery,Vietnamese Restaurant,Italian Restaurant,French Restaurant,Diner
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,College Quad,Bookstore,College Gym,Coffee Shop,Café,Gym / Fitness Center,Trail,Bus Line,Park,Eastern European Restaurant


## Visualize the resulting clusters

In [161]:
# create map

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Notes
The approach used was to sub-select neighborhoods down to those containing Toronto, as advised.
The fact that the initial selection was done with postcodes introduced some changes from the original approach in the example worksheet.
Rather than grouping the districts down to the five that contain Toronto the postcode was kept as an indicator of neighborhood. Thsi was so
that the clustering had more than five districts to work with. Sub-slecting on Toronto and keeping postcodes allowed 38 locations to be explored.
Having a 500m radius ofr venue selection yielded very low values. This confounded teh clustering which needed more data to work with. Hence, the
radius was increased to 1000m which gave better numbers of venues. This indicates that Toronoto probably has a more sparse distribution of venues
than New York.
In generating the final data frame the join required changing the postcode heading in the original frame to neighborhoods so that the final join 
worked. The map above shows the 38 districts, clustered on venue types within a 1000m radius of teh postcode. The 38 posctodes were selected for having Toronto in them.
